In [89]:
import data_ingestion
import os
import pandas as pd
from typing import Any

# Define constants for dataset details and paths
DATASET_ORIGIN = 'rashikrahmanpritom'
DATASET_NAME = 'data-science-job-posting-on-glassdoor'
DOWNLOAD_PATH = 'data'

# Call the function to download and unzip the dataset
# data_ingestion.download_and_unzip_kaggle_dataset(DATASET_ORIGIN, DATASET_NAME, DOWNLOAD_PATH)

csv_file_path = os.path.join(DOWNLOAD_PATH, 'Uncleaned_DS_Jobs.csv')
df = data_ingestion.load_csv(csv_file_path)

CSV file 'data\Uncleaned_DS_Jobs.csv' loaded successfully.


In [90]:
def normalize_column_names(df):
    """
    Convert DataFrame column names to lowercase and replace spaces with underscores.
    This function might break YAGNI rule but it's defined in case more complex rules are needed for normalization.
    
    Parameters:
        df (pd.DataFrame): The DataFrame with original column names.
        
    Returns:
        pd.DataFrame: The DataFrame with normalized column names.
    """
    df.columns = df.columns.str.lower().str.replace(' ', '_')
    return df

df = normalize_column_names(df)

In [91]:
df = df.drop('index', axis=1)
df = df.drop('job_description', axis=1)

In [92]:
# Create a new column with the text proceeding the newline character
# df[['company_name', 'company_star_rating']] = df['company_name'].str.split('\n', expand=True)
df['company_name'] = df['company_name'].str.split('\n').str[0]

In [93]:
df['salary_estimate'] = df['salary_estimate'].str.extract(r'(\$\d+K-\$\d+K)')
# df[['salary_min', 'salary_max']] = df['salary_estimate'].str.extract(r'\$(\d+)K-\$(\d+)K')
# df['salary_min'] = df['salary_min'].astype(int) * 1000
# df['salary_max'] = df['salary_max'].astype(int) * 1000

# df[['salary_min', 'salary_max']] = df['Salary Estimate'].str.extract(r'\$(\d+)[Kk]-\$(\d+)[Kk]')
# df['salary_min'] = df['salary_min'].astype(int) * 1000
# df['salary_max'] = df['salary_max'].astype(int) * 1000

In [10]:
# df['location_city'] = df['Location'].str.split(',').str[0]
# df['location_state'] = df['Location'].str.split(',').str[1].str.strip()

# df['headquarters_city'] = df['Headquarters'].str.split(',').str[0]
# df['headquarters_state'] = df['Headquarters'].str.split(',').str[1].str.strip()

# df['location_city'] 
# df['location_state']
# df['headquarters_city']
# df['headquarters_state']

In [94]:
def replace_vals_in_cols(df: pd.DataFrame, cols: list[str], old_val: Any, new_val: Any) -> pd.DataFrame:
    """
    Replace occurrences of `old_val` with `new_val` in specified columns of the DataFrame.

    Args:
        df (pd.DataFrame): The DataFrame to modify.
        cols (list[str]): A list of column names in which to perform the replacement.
        old_val (Any): The value to be replaced.
        new_val (Any): The value to replace `old_val` with.

    Returns:
        pd.DataFrame: The DataFrame with the specified values replaced.
    """
    for col in cols:
        df[col] = df[col].replace(old_val, new_val)
    
    return df

cols_list = ['rating', 'headquarters', 'size', 'founded', 'type_of_ownership', 'industry', 'sector', 'revenue', 'competitors']
df = replace_vals_in_cols(df, cols_list, ['-1', -1, -1.0], pd.NA)